In [1]:
import matlab.engine
import pandas
from keras.models import load_model
import random
import logging

logging.basicConfig(level=logging.INFO)

Importowanie wymaganych autorskich bibliotekfunkcji do przetwarzania danych

In [2]:

%run read_data_and_prepare_to_training.ipynb
%run load_models.ipynb

ModuleNotFoundError: No module named 'tsai'

ModuleNotFoundError: No module named 'tsai'

## Parametry uszkodzeń
W kolejnej komórce należy wprowadzić sprawdzane parametry uszkodzeń, gdzie:
- **SDrift** wyznacza przesunięcie dryftu czujnika <-1 ; 1>
- **SGain** jest to parametr wzmocnienia zwarcia zęba przekładni <-2 ; 0>
- **ShaftWear** jest to parametr zużycia wału. Może on przyjmować 2 wartości 0 oznacza brak uszkodzenia oraz -1 oznaczający zużycie wału

In [2]:
SDrift = -0.8
SGain = 0
ShaftWear = 0

In [3]:
eng = matlab.engine.start_matlab()
correct = eng.SimulateTransmissionCasing(SDrift,SGain,ShaftWear)
eng.exit()

if not correct:
    logger.error("Execution of simulation was incorrect")
    raise SystemExit("Simulink error!")

SyntaxError: invalid syntax (<ipython-input-3-ac4e5865b2f0>, line 5)

### Read simulation data from file 

In [4]:
simulation_data = pandas.read_parquet("SimulationData.parquet").to_pandas()
simulation_data

### Simulation parameters required for converting data

In [ ]:
number_of_columns_in_one_simulation = 11
time_sample_for_data = 0.01
resolution = 2

### Convert output data from simulation to proper model input

In [6]:
logger.info("Converting time columns from data time to float")
train = simulation_data_time_to_float(simulation_data, number_of_columns_in_one_simulation)
logger.info("Converting tacho columns from impulses to angular velocity")
train = add_angular_velocity_calculated_from_pulses(train, number_of_columns_in_one_simulation, time_sample_for_data,
                                                    resolution)
logger.info("Reshape and prepare data to train")
X,_ = prepare_data_to_train(train, number_of_columns_in_one_simulation)

In [7]:
simulation_data = pandas.read_parquet("SimulationData.parquet").to_pandas()
simulation_data

,vibration,tacho1,tacho2
0,-0.632228,0.0,0.0
1,-1.351946,0.0,0.0
2,-1.426686,0.0,0.0
3,-0.618158,0.0,0.0
4,0.488226,0.0,0.0
...,...,...,...
295,0.482954,0.0,0.0
296,-0.624341,0.0,0.0
297,-1.428926,0.0,0.0
298,-1.349077,0.0,0.0


In [8]:
model = load_model('PredictFaultData')

In [9]:
X_simulation = simulation.values

In [10]:
n_signals, n_steps, n_length = 3, 20, 15
X_simulation = X_simulation.reshape((1, n_steps, n_length, n_signals))

In [11]:
X_simulation

array([[[[-0.63222814,  0.        ,  0.        ],
         [-1.35194628,  0.        ,  0.        ],
         [-1.42668598,  0.        ,  0.        ],
         [-0.61815778,  0.        ,  0.        ],
         [ 0.48822567,  0.        ,  0.        ],
         [ 1.01488421,  0.        ,  0.        ],
         [ 0.59277408,  0.        ,  0.        ],
         [-0.3041332 ,  0.        ,  0.        ],
         [-0.79771761,  0.        ,  0.        ],
         [-0.41460483,  0.        ,  0.        ],
         [ 0.47814046,  0.        ,  0.        ],
         [ 1.01083886,  0.        ,  0.        ],
         [ 0.61215576,  0.        ,  0.        ],
         [-0.46273747,  0.        ,  0.        ],
         [-1.36216561,  0.        ,  0.        ]],

        [[-1.4149876 ,  0.        ,  0.        ],
         [-0.74100656,  0.        ,  0.        ],
         [-0.13420017,  0.        ,  0.        ],
         [-0.32631536,  0.        ,  0.        ],
         [-1.30765478,  0.        ,  0.        ]

In [12]:
predictions = model.predict(X_simulation)

In [13]:
model_SensorDrift = load_model('PredictFaultData_SensorDrift');
model_ShaftWear = load_model('PredictFaultData_ShaftWear');
model_ToothFault = load_model('PredictFaultData_ToothFault');

In [14]:
prediction_SensorDrift = model_SensorDrift.predict(X_simulation)
prediction_ShaftWear = model_ShaftWear.predict(X_simulation)
prediction_ToothFault = model_ToothFault.predict(X_simulation)

In [15]:
prediction_SensorDrift

array([[0.59930885]], dtype=float32)

In [16]:
prediction_ShaftWear

array([[0.08464664]], dtype=float32)

In [17]:
prediction_ToothFault

array([[0.00020415]], dtype=float32)

In [18]:
list_prediction_SensorDrift = prediction_SensorDrift.tolist()
list_prediction_ShaftWear = prediction_ShaftWear.tolist()
list_prediction_ToothFault = prediction_ToothFault.tolist()

In [19]:
list_prediction_ToothFault[0][0]

0.0002041459083557129

In [20]:
threshpreds = [(list_prediction_SensorDrift[0][0] > 0.4) * 1,(list_prediction_ShaftWear[0][0] > 0.4) * 1, (list_prediction_ToothFault[0][0] > 0.4) * 1]

In [21]:
threshpreds

[1, 0, 0]